---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="bcc96043" -->
# Table des matières
1. [Le paradoxe de l'exactitude](#le-paradoxe-de-lexactitude)
1. [Avoir $99~\%$ au test sans étudier!](#avoir-99-au-test-sans-étudier)
1. [Tactiques éprouvées pour combattre le débalancement des classes](#tactiques-éprouvées-pour-combattre-le-débalancement-des-classes)
  1. [Collecter plus de données!!!](#collecter-plus-de-données)
  1. [Changer de métrique de performance](#changer-de-métrique-de-performance)
  1. [Ré-échantillonner le jeu de données](#ré-échantillonner-le-jeu-de-données)
  1. [Générer des échantillons synthétiques](#générer-des-échantillons-synthétiques)
  1. [Essayer les modèles pénalisés](#essayer-les-modèles-pénalisés)
1. [Exemples de ré-échantillonnage](#exemples-de-ré-échantillonnage)
    1. [Lecture d'un jeu de données déséquilibré contenant deux classes](#lecture-dun-jeu-de-données-débalancé-contenant-deux-classes)
    1. [Le piège des métriques](#le-piège-des-métriques)
    1. [Techniques de bases en ré-échantillonnage](#techniques-de-bases-en-ré-échantillonnage)
      1. [Sous-échantillonnage aléatoire](#sous-échantillonnage-aléatoire)
      1. [Suréchantillonnage aléatoire](#suréchantillonnage-aléatoire)
    1. [Techniques plus avancées en ré-échantillonnage](#techniques-plus-avancées-en-ré-échantillonnage)
      1. [Génération d'un nouveau jeu de données](#génération-dun-nouveau-jeu-de-données)
      1. [Sous-échantillonnage aléatoire avec apprentissage déséquilibré](#sous-échantillonnage-aléatoire-avec-apprentissage-déséquilibré)
      1. [Suréchantillonnage aléatoire avec apprentissage déséquilibré](#suréchantillonnage-aléatoire-avec-apprentissage-déséquilibré)
      1. [Sous-échantillonnage avec liens Tomek](#sous-échantillonnage-avec-liens-tomek)
      1. [Sous-échantillonnage avec centroïdes de regroupements](#sous-échantillonnage-avec-centroïdes-de-regroupements)
      1. [Suréchantillonnage avec SMOTE](#suréchantillonnage-avec-smote)
      1. [Suréchantillonnage suivi d'un sous-échantillonnage](#suréchantillonnage-suivi-dun-sous-échantillonnage)
1. [Pour en savoir plus](#pour-en-savoir-plus)
<!-- #endregion -->



In [ ]:
import imblearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import (ClusterCentroids, RandomUnderSampler,
                                     TomekLinks)
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

seed = 42
np.random.seed(seed)



<!-- #region id="5054b330" -->
# <a id=le-paradoxe-de-lexactitude>Le paradoxe de l'exactitude</a>
<!-- #endregion -->

<!-- #region id="f29c7e62" -->
En apprentissage automatique l'[exactitude](https://fr.wikipedia.org/wiki/Exactitude_et_précision) (*accuracy*)
est une mesure de qualité en classification. Il existe plusieurs autres mesures de qualité; elles sont présentées
dans le module sur les métriques de qualité en classification. L'exactitude est la métrique la plus intuitive de toutes.
Elle représente la proportion de prédictions correctes (positives et négatives) parmi l'ensemble des prédictions.

Le [paradoxe de l'exactitude](https://en.wikipedia.org/wiki/Accuracy_paradox) est le nom de la situation où vos mesures d'exactitude indiquent que vous avez une excellente exactitude (telle que $90~\%$), mais que l'exactitude ne reflète que la distribution de classe majoritaire sous-jacente.

C'est un problème très courant, car l'exactitude est souvent la première mesure utilisée pour évaluer nos modèles dans les problèmes de classification.
<!-- #endregion -->

<!-- #region id="a71d5839" -->
# <a id=avoir-99-au-test-sans-étudier>Avoir $99~\%$ au test sans étudier!</a>
<!-- #endregion -->

<!-- #region id="75538af5" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/machine-learning-monitoring.png"  width="500" />
    <div>
    <font size="1.5">Image Source: https://evidentlyai.com/blog/machine-learning-monitoring-what-it-is-and-how-it-differs/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="578975c1" -->
Que se passe-t-il dans les modèles lorsque l'entraînement est fait sur un jeu de données déséquilibré?

Lorsqu'une exactitude de $99~\%$ est obtenue avec des données déséquilibrées (avec $99~\%$ des instances dans la « classe 1 »),
c'est parce que les modèles examinent les données et décident intelligemment que la meilleure chose à faire est
de toujours prédire « classe 1 » pour atteindre une grande exactitude.

Est-ce si mauvais que ça? **Oui!** Pensez-y. Dans la pratique, ce sont les instances de la classe rarissime qui nous
intéressent réellement. Le classificateur précédent ne pourrait en détecter aucune dans un jeu de données ne
contenant que des instances rarissimes!

Comment régler ce problème ?
<!-- #endregion -->

<!-- #region id="610c8e8a" -->
# <a id=tactiques-éprouvées-pour-combattre-le-débalancement-des-classes>Tactiques éprouvées pour combattre le débalancement des classes</a>
<!-- #endregion -->

<!-- #region id="3bec506c" -->
## <a id=collecter-plus-de-données>Collecter plus de données!!!</a>
<!-- #endregion -->

<!-- #region id="6cb7bfca" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/more-data-illustration.jpeg"  width="500" />
    <div>
    <font size="1.5">Image Source: https://technofaq.org/posts/2020/06/3-ways-that-technology-improves-productivity/</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region colab_type="text" id="b89f60d9" -->
Vous pensez peut-être que c'est idiot, mais la collecte de plus de données est presque toujours négligée.

Pouvez-vous collecter plus de données? Prenez une seconde et demandez-vous si vous êtes capable de collecter plus de données sur votre problème.

Un ensemble de données plus volumineux pourrait exposer une perspective différente et peut-être plus équilibrée des classes.
<!-- #endregion -->

<!-- #region id="8b73b21c" -->
## <a id=changer-de-métrique-de-performance>Changer de métrique de performance</a>
<!-- #endregion -->

<!-- #region id="9b68f2f2" -->
L'exactitude n'est pas la métrique à utiliser lorsque vous travaillez avec un jeu de données déséquilibrées.
Nous avons vu que c'est trompeur.

D'autres métriques ont été conçues pour vous raconter une histoire plus véridique lorsque vous travaillez
avec des classes déséquilibrées. En voici quelques-unes présentées dans le module sur les métriques de
qualité en classification:

- exactitude,
- précision,
- rappel,
- mesure $F_1$ (ou F-score),
- aire sous les courbes ROC et Précision-Rappel.

<!-- #endregion -->

<!-- #region id="5c3c8d0a" -->
## <a id=ré-échantillonner-le-jeu-de-données>Ré-échantillonner le jeu de données</a>
<!-- #endregion -->

<!-- #region id="f4e7901e" -->
Il est également possible de modifier un jeu de données afin d'obtenir des classes plus équilibrées. Cela
mène souvent à une amélioration des performances en classification.

Cette modification s'appelle le
[ré-échantillonnage](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis).
Il existe deux méthodes principales pour uniformiser les classes:
- l'ajout de copies d'instances de la classe sous-représentée : suréchantillonnage (*oversampling*),
- la suppression d'instances de la classe surreprésentée : sous-échantillonnage (*undersampling*).

Ces approches sont souvent très faciles à mettre en œuvre et rapides à exécuter. Elles sont un excellent point de départ.

En fait, il vaut mieux essayer les deux approches sur tous les jeux de données déséquilibrés, juste pour voir si cela donne une amélioration de vos mesures préférées.

Voici quelques règles de base :
- le sous-échantillonnage est préférable lorsque vous avez beaucoup de données (des dizaines, des centaines de milliers d'instances ou plus),
- le suréchantillonnage est préférable lorsque vous n’avez pas beaucoup de données (des dizaines de milliers d’enregistrements ou moins),
- tester différents ratios de rééchantillonnage (il n'est pas toujours nécessaire d'avoir un ratio de 1:1 dans un problème de classification binaire).
<!-- #endregion -->

<!-- #region id="6401987b" -->
## <a id=générer-des-échantillons-synthétiques>Générer des échantillons synthétiques</a>
<!-- #endregion -->

<!-- #region id="17564c8c" -->
Il existe des algorithmes systématiques utilisables pour générer des échantillons synthétiques.
Le plus populaire de ces algorithmes est appelé SMOTE ou technique de suréchantillonnage minoritaire synthétique.

SMOTE est une méthode de suréchantillonnage. Elle fonctionne en créant de nouveaux échantillons synthétiques de
la classe mineure au lieu d'en créer des copies.

Il existe plusieurs implémentations de cet algorithme. La librairie Python
[imbalanced-learn](https://imbalanced-learn.org/stable/user_guide.html) en fournit un certain nombre
ainsi que diverses autres techniques de ré-échantillonnage.
<!-- #endregion -->

<!-- #region id="43ead402" -->
## <a id=essayer-les-modèles-pénalisés>Essayer les modèles pénalisés</a>
<!-- #endregion -->

<!-- #region id="c1ee898b" -->
La classification pénalisée impose un coût supplémentaire au modèle pour faire des erreurs de classification
à la classe minoritaire pendant l'entraînement. Ces pénalités peuvent inciter le modèle à accorder plus
d’attention à la classe minoritaire. C'est une approche utilisée dans les réseaux de neurones.

Il existe des outils facilitant l'emploi de modèles pénalisés. Par exemple, Weka
peut assigner à tout classificateur une matrice de pénalités personnalisée pour la classification des échecs.

L’utilisation de la pénalisation est souhaitable si vous êtes restreints à un algorithme précis et êtes
incapables de rééchantillonner ou si vous obtenez des résultats médiocres.
<!-- #endregion -->

<!-- #region _cell_guid="e38ddf38-d027-4eae-95c8-749f8f40db43" _uuid="1e287038acf96fc6d6825e77ba97b03f0824be0a" colab_type="text" id="5ce03805" -->
# <a id=exemples-de-ré-échantillonnage>Exemples de ré-échantillonnage</a>
<!-- #endregion -->

<!-- #region id="b0bc04c4" -->
## <a id=lecture-dun-jeu-de-données-débalancé-contenant-deux-classes>Lecture d'un jeu de données déséquilibré contenant deux classes</a>
<!-- #endregion -->

<!-- #region id="5c3086b3" -->
Le jeu contient $100\,000$ données de 20 variables, c'est-à-dire, $X=[x_{1}, \cdots , x_{20}]$.
La classe majoritaire (c.-à-d. la classe « 0 ») contient $99~\%$ des données et la classe minoritaire (c.-à-d. la classe « 1 ») contient les $1~\%$ restants.
<!-- #endregion -->



In [ ]:
df = pd.read_csv("/pax/shared/GIF-U014/dataset_debalance.csv")
y = df["target"]
X = df.drop("target", axis=1).to_numpy()


In [ ]:
# Génération d'un histogramme des valeurs de la réponse $y$

target_count = df.target.value_counts()
print("Classe 0:", target_count[0])
print("Classe 1:", target_count[1])
print("Proportion:", round(target_count[0] / target_count[1], 4), ": 1")
ax = df.target.value_counts().plot(kind="bar", title="Count (target)", rot=0)
ax.set_xlabel("Classe", fontsize=15)
ax.set_ylabel("N", fontsize=15, rotation=0)



<!-- #region _cell_guid="c91f9c5d-05d2-478e-9dfb-1cb848bc0fe4" _uuid="8683656d1bfdef6b5c0c623597dcbc4160a0edc1" colab_type="text" id="07ffc549" -->
## <a id=le-piège-des-métriques>Le piège des métriques</a>
<!-- #endregion -->

<!-- #region id="a411ea48" -->
L'un des principaux problèmes auxquels se heurtent les utilisateurs novices lorsqu'ils traitent des jeux
de données déséquilibrés est lié aux métriques utilisées pour évaluer leur modèle. Utiliser des métriques
plus simples comme l'exactitude peut être trompeur. Dans un ensemble de données avec des
classes très déséquilibrées, si le classificateur "prédit" toujours la classe la plus courante sans
effectuer d'analyse des caractéristiques, il conservera une exactitude élevée, évidemment illusoire.

Faisons cette expérience en utilisant une simple validation croisée sans modifier les caractéristiques. 

> À noter que les principes d'utilisation de la validation croisée sont présentés dans un des modules de la méthodologie.

Dans ce qui suit, on ne normalise pas les données, car elles sont déjà toutes du même ordre de grandeur.
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

# Sélection du classificateur
model = SVC(kernel="linear")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))



<!-- #region _cell_guid="c46b98a7-d500-4911-a7b5-c8fc8fbed069" _uuid="5b17ee8d3629cc346398e63269205e5b654cef80" colab_type="text" id="524fc8da" -->
Exécutons maintenant le même code, mais en n'utilisant qu'une seule caractéristique parmi les 20 que contient chaque donnée. Cela devrait considérablement réduire l'exactitude du classificateur puisqu'il y a une grande perte d'information permettant de séparer les nuages de points de chaque classe. Est-ce le cas?
<!-- #endregion -->



In [ ]:
model = SVC(kernel="linear")
model.fit(X_train[:, 0:1], y_train)
y_pred = model.predict(X_test[:, 0:1])

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))



<!-- #region _cell_guid="68d4ffaf-4412-4ff3-a9cd-38bea195da3b" _uuid="d9d8b57f458bdd107ae08b76b0008d80e0674d97" colab_type="text" id="66957cdb" -->
Comme nous pouvons le constater, le taux de précision élevé n’était qu’illusion. De cette manière, le choix de la métrique utilisée dans les jeux de données non équilibrés est extrêmement important.
<!-- #endregion -->

<!-- #region _cell_guid="b27346eb-7bf3-4360-993a-fb91e62bb937" _uuid="875f5ab3b5afcdaf3c7754ce957cb01fd32bf65c" colab_type="text" id="38045e63" -->
## <a id=techniques-de-bases-en-ré-échantillonage>Techniques de bases en ré-échantillonnage</a>
<!-- #endregion -->

<!-- #region id="307de623" -->
Le ré-échantillonnage est une technique largement adoptée pour traiter les jeux de données très déséquilibrés.
Cela consiste à retirer des échantillons de la classe majoritaire (sous-échantillonnage ou
*undersampling*) ou à ajouter d'autres exemples dans la classe minoritaire (suréchantillonnage ou *oversampling*).

<!-- #endregion -->

<!-- #region _cell_guid="03d31a16-7b66-4096-88d7-d548db734390" _uuid="2232ac0fb192a468486b400846f88913a36957e6" colab_type="text" id="64f722bd" -->
![](https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/resampling.png)
<!-- #endregion -->

<!-- #region _cell_guid="0df1a3f3-49ff-4ada-80f1-198bbbd79525" _uuid="67e203e0919c818e871650ef194fe497df2d39b5" colab_type="text" id="80595c90" -->
Malgré l'avantage de l'équilibrage des classes, ces techniques ont aussi leurs faiblesses. La
mise en œuvre la plus simple du suréchantillonnage consiste à dupliquer aléatoirement des données de la
classe minoritaire, ce qui peut entraîner un surapprentissage (*overfitting*). Dans le
sous-échantillonnage, la technique la plus simple consiste à supprimer aléatoirement des enregistrements
de la classe majoritaire, ce qui entraîne nécessairement une perte d'information.

Implémentons un exemple de base, qui utilise la méthode `DataFrame.sample` pour obtenir aléatoirement
des échantillons de chaque classe:
<!-- #endregion -->



In [ ]:
# Nombre de valeurs par classe
count_class_0, count_class_1 = df.target.value_counts()

# Séparation des classes
df_class_0 = df[df["target"] == 0]
df_class_1 = df[df["target"] == 1]



<!-- #region _cell_guid="152ea73a-aa23-4fd2-a3f5-1f55c69041ae" _uuid="b765be76a182930feb650b01dd4d1de90501bbce" colab_type="text" id="9cfc781e" -->
### <a id=sous-échantillonnage-aléatoire>Sous-échantillonnage aléatoire</a>
<!-- #endregion -->



In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print("Sous-échantillonnage aléatoire:")
print(df_test_under.target.value_counts())

ax = df_test_under.target.value_counts().plot(kind="bar", title="Count (target)", rot=0)
ax.set_xlabel("Classe", fontsize=15)
ax.set_ylabel("N", fontsize=15, rotation=0)



<!-- #region _cell_guid="be656d47-e529-4533-b975-cf6de072d959" _uuid="17192fe8557463941e3abd4633b58ced0037721b" colab_type="text" id="79ffd335" -->
### <a id=suréchantillonnage-aléatoire>Suréchantillonnage aléatoire</a>
<!-- #endregion -->



In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print("Suréchantillonnage aléatoire:")
print(df_test_over.target.value_counts())

ax = df_test_over.target.value_counts().plot(kind="bar", title="Count (target)", rot=0)
ax.set_xlabel("Classe", fontsize=15)
ax.set_ylabel("N", fontsize=15, rotation=0)



<!-- #region _cell_guid="9fd90ddc-f2fc-487d-b177-0c540daf2eff" _uuid="9672899d4029b71b72897927ce464d6d7427ce77" colab_type="text" id="2ca29e2a" -->
## <a id=techniques-plus-avancées-en-ré-échantillonage>Techniques plus avancées en ré-échantillonnage</a>
<!-- #endregion -->

<!-- #region id="e38187f2" -->
Un certain nombre de techniques de ré-échantillonnage plus sophistiquées ont été proposées dans la
littérature scientifique.

Par exemple, il est possible de regrouper les données de la classe majoritaire et d'effectuer le
sous-échantillonnage en supprimant des données de chaque regroupement (*cluster*). On préserve
ainsi la distribution multidimensionnelle des données tout en réduisant leur nombre.
En suréchantillonnage, au lieu de créer des copies exactes des données de la classe de minorité,
nous pouvons introduire de petites variations dans ces copies, créant ainsi des échantillons
synthétiques plus divers.

Appliquons certaines de ces techniques de ré-échantillonnage en utilisant la librairie Python
[imbalanced-learn](https://imbalanced-learn.org/stable/auto_examples/index.html#general-examples/).
Elle est compatible avec Scikit-learn et fait partie des projets contributeurs de Scikit-learn.

<!-- #endregion -->

<!-- #region _cell_guid="5542beb4-6aee-401a-8bc0-2a522fbbe90b" _uuid="ff93d1707c416178c010c125319220b785dca984" colab_type="text" id="93b42a56" -->
### <a id=génération-dun-nouveau-jeu-de-données>Génération d'un nouveau jeu de données</a>
<!-- #endregion -->

<!-- #region id="b13eaa80" -->
Pour faciliter la visualisation, créons un nouveau jeu de données de taille réduite, toujours avec
deux classes, et ne contenant plus que 100 données $X$ de 20 variables. Cette fois-ci, la répartition
des classes est de ($90~\%$, $10~\%$).

À nouveau, on ne normalise pas les données, car elles sont déjà toutes du même ordre de grandeur.
<!-- #endregion -->



In [ ]:
X, y = make_classification(
    n_classes=2,
    class_sep=1.5,
    weights=[0.9, 0.1],
    n_informative=3,
    n_redundant=1,
    flip_y=0,
    n_features=20,
    n_clusters_per_class=1,
    n_samples=100,
    random_state=10,
)

df = pd.DataFrame(X)
df["target"] = y
ax = df.target.value_counts().plot(kind="bar", title="Count (target)", rot=0)
ax.set_xlabel("Classe", fontsize=15)
ax.set_ylabel("N", fontsize=15, rotation=0)



<!-- #region _cell_guid="4de57657-3aed-4444-a609-318063391763" _uuid="d348b114ec1594eeefa0a67aab8b54e5b9ee2bdf" colab_type="text" id="b18e6bf1" -->
Nous allons également créer une fonction de tracé, <code>plot_2d_space</code>, pour afficher la répartition des données.
<!-- #endregion -->



In [ ]:
def plot_2d_space(X, y, label="Classes"):
    colors = ["#1F77B4", "#FF7F0E"]
    markers = ["o", "s"]
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    for l, c, m in zip(np.unique(y), colors, markers):
        plt.scatter(X[y == l, 0], X[y == l, 1], c=c, label=l, marker=m, alpha=0.5)
    ax.set_xlabel("$PCA_{1}$", fontsize=15)
    ax.set_ylabel("$PCA_{2}$", fontsize=15)
    plt.title(label)
    plt.legend(loc="upper right")
    plt.show()



<!-- #region _cell_guid="2c565bbc-39ed-45a2-8b3d-bc501e2510aa" _uuid="aabc110dd8d1b36345df6aada1c59b864e48e8e6" colab_type="text" id="7cfbf177" -->
Étant donné que le jeu de données est en 20-D et que nos graphiques seront en 2-D, la taille du jeu de données est
réduite au moyen de l'analyse en composantes principales (*PCA*). On n'utilise que les deux
premières composantes principales.
<!-- #endregion -->



In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(X)

plot_2d_space(
    X,
    y,
    "Jeu de données non équilibré \n (\
2 premières composantes principales)",
)



<!-- #region _cell_guid="c3c9a24f-3cd0-4c8d-8a4d-4403c4f1a641" _uuid="0d7316b04837aa103003d667f63ecb05d43fc04e" colab_type="text" id="2d9db88c" -->
### <a id=sous-échantillonnage-aléatoire-avec-apprentissage-déséquilibré>Sous-échantillonnage aléatoire avec apprentissage déséquilibré</a>

Réduire le nombre de données de la classe majoritaire n'est pas une bonne option dans cet exemple, car il ne reste que peu de données à utiliser par la suite.
<!-- #endregion -->



In [ ]:
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_resample(X, y)
id_rus = rus.sample_indices_

print("Indices retirés :", id_rus)

plot_2d_space(X_rus, y_rus, "Sous-échantillonnage aléatoire")



<!-- #region _cell_guid="c3c9a24f-3cd0-4c8d-8a4d-4403c4f1a641" _uuid="0d7316b04837aa103003d667f63ecb05d43fc04e" colab_type="text" id="2d9db88c" -->
### <a id=suréchantillonnage-aléatoire-avec-apprentissage-déséquilibré>Suréchantillonnage aléatoire avec apprentissage déséquilibré</a>

Augmenter le nombre de données de la classe minoritaires n'est pas une bonne option dans cet exemple, car il y a trop peu de données différentes.
<!-- #endregion -->



In [ ]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

print(X_ros.shape[0] - X.shape[0], "nouveaux points aléatoires")

plot_2d_space(X_ros, y_ros, "Suréchantillonnage aléatoire")



<!-- #region _cell_guid="3156134f-539b-48b8-b9d0-64095fe50c1c" _uuid="b3f9ac47a157d9096a626360408859b795299c24" colab_type="text" id="60e05b4b" -->
### <a id=sous-échantillonnage-avec-liens-tomek>Sous-échantillonnage avec liens Tomek</a>
<!-- #endregion -->

<!-- #region id="4c3257c9" -->
Les liens Tomek sont des paires d'instances très proches, mais de classes opposées. La suppression des occurrences de la classe majoritaire de chaque paire augmente l'espace entre les deux classes, facilitant ainsi le processus de classification.
<!-- #endregion -->

<!-- #region _cell_guid="f131f7e0-007d-406e-9e1c-db352d2d8433" _uuid="85c01c0e6baa1b984585c1db34c5ab0315cbf8ff" colab_type="text" id="d686c797" -->
![](https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/tomek.png?v=2)
<!-- #endregion -->

<!-- #region _cell_guid="45bf057b-369c-4f37-a5ac-7417ad79253d" _uuid="733a86ccfaaabf701fb2d1f9997c732b19630df3" colab_type="text" id="cc8ec7ff" -->
Dans le code ci-dessous, nous utiliserons `sampling_strategy = 'majority'` pour ré-échantillonner la classe majoritaire.
<!-- #endregion -->



In [ ]:
tl = TomekLinks(sampling_strategy="majority")
X_tl, y_tl = tl.fit_resample(X, y)
id_tl = tl.sample_indices_

print("Indices retirés:", id_tl)

plot_2d_space(X_tl, y_tl, " Sous-échantillonnage des liens Tomek")



<!-- #region _cell_guid="fef831bd-ecec-429c-aef2-5d51d1188820" _uuid="b4e75fffe4c91afcd63705aa7bcb16b6fd9f6b1f" colab_type="text" id="a56a69b6" -->
### <a id=sous-échantillonnage-avec-centroïdes-de-regroupements>Sous-échantillonnage avec centroïdes de regroupements</a>
<!-- #endregion -->

<!-- #region id="8519b43a" -->
Cette technique effectue un sous-échantillonnage en générant des centroïdes basés sur des méthodes
de regroupement (*clustering methods*). Les données seront préalablement regroupées par similarité,
afin de préserver les informations.

Dans cet exemple, le dictionnaire `{0: 10}` pour le paramètre `sampling_strategy`, indique
de préserver 10 éléments de la classe majoritaire (0) et conserver toutes les données de la classe minoritaire (1).
<!-- #endregion -->



In [ ]:
cc = ClusterCentroids(sampling_strategy={0: 10})
X_cc, y_cc = cc.fit_resample(X, y)

plot_2d_space(X_cc, y_cc, "Cluster Centroids under-sampling")



<!-- #region _cell_guid="78adb7b4-a7e1-4d10-9cc2-6bf6477c63df" _uuid="b3741f5c14acdbd76e25725e2d73df5f2cb0a239" colab_type="text" id="b19a591d" -->
### <a id=suréchantillonnage-avec-smote>Suréchantillonnage avec SMOTE</a>
<!-- #endregion -->

<!-- #region id="902c694e" -->
SMOTE (*synthetic minority oversampling technique*) consiste à synthétiser des éléments pour
la classe minoritaire, sur la base de ceux qui existent déjà. Cela fonctionne de manière aléatoire
en sélectionnant un point de la classe minoritaire et en calculant les k-voisins les plus proches
pour ce point (k-NN). Les points synthétiques sont ajoutés entre le point choisi et ses voisins.

<!-- #endregion -->

<!-- #region _cell_guid="5162646f-da82-4877-b6d8-25a8be7f42e9" _uuid="51697e21b7cdb4064dda18aa24e6ecf039b1132b" colab_type="text" id="81102cbc" -->
 ![](https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/smote.png)
<!-- #endregion -->

<!-- #region _cell_guid="9c0a0d78-8427-437e-aa24-ffe2bd12edb2" _uuid="9393851db694c178faf93615bf05addedf5d678b" colab_type="text" id="c6d4e26e" -->
Nous allons utiliser `sampling_strategy = 'minority'` pour ré-échantillonner la classe minoritaire.
<!-- #endregion -->



In [ ]:
smote = SMOTE(sampling_strategy="minority")
X_sm, y_sm = smote.fit_resample(X, y)

plot_2d_space(X_sm, y_sm, "Suréchantillonnage SMOTE")



<!-- #region _cell_guid="20c3cdbb-a7c1-45a5-8dd9-84cff7d9af31" _uuid="d7ebbeb741ad6469cb7ebbced3499acd3c49856a" colab_type="text" id="69e1afe4" -->
### <a id=suréchantillonnage-suivi-dun-sous-échantillonnage>Suréchantillonnage suivi d'un sous-échantillonnage</a>
<!-- #endregion -->

<!-- #region id="92988821" -->
Nous pouvons maintenant combiner le suréchantillonnage et le sous-échantillonnage, en utilisant les techniques de liens SMOTE et Tomek.
<!-- #endregion -->



In [ ]:
smt = SMOTETomek(sampling_strategy="auto")
X_smt, y_smt = smt.fit_resample(X, y)

plot_2d_space(X_smt, y_smt, "SMOTE + Tomek links")



<!-- #region id="789b0f04" -->
# <a id=pour-en-savoir-plus>Pour en savoir plus</a>
<!-- #endregion -->

<!-- #region _cell_guid="28d4431f-bc32-4cf0-988e-9df412cc22c1" _uuid="401bcb8e508e584feca3a34ecfb9de270fde951c" colab_type="text" id="4ea87a9d" -->
- [Documentation](https://imbalanced-learn.org/stable/user_guide.html) de la librairie *imbalanced-learn*:
- [Chawla, Nitesh V., et al. "SMOTE: synthetic minority over-sampling technique." Journal of artificial intelligence research 16 (2002)](http://export.arxiv.org/pdf/1106.1813)
<!-- #endregion -->
